In [3]:
import torchvision
import torch
from torch import nn, optim
from pathlib import Path
from anomalib.data import Multi_Category_MVTec

In [4]:
CATEGORY_NAME = ["bottle","cable","capsule","carpet","grid","hazelnut","leather","metal_nut","pill","screw","tile","toothbrush","transistor","wood","zipper"]

In [5]:
model = torchvision.models.mobilenet_v2(pretrained=True)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(in_features=1280, out_features=15, bias=True)
)
if torch.cuda.is_available():
    model.cuda()

In [6]:
optimizer = optim.SGD(model.parameters(),lr=0.1,momentum=0.9)
lr_scheduler = optim.lr_scheduler.StepLR(
    optimizer=optimizer, step_size=2, gamma=0.99)
loss = nn.CrossEntropyLoss()

In [7]:
datamodule = Multi_Category_MVTec(
    root="../datasets/MVTec",
    category=CATEGORY_NAME,
    image_size=224,
    train_batch_size=32,
    test_batch_size=32,
    num_workers=4,
    task="category_classification"
)
datamodule.prepare_data()
datamodule.setup()

E:\PythonProjects\anomaly_detection\anomalib\data\multi_category_mvtec.py:227: UserWarning: seed is None. When seed is not set, images from the normal directory are split between training and test dir. This will lead to inconsistency between runs.
  warnings.warn(
E:\PythonProjects\anomaly_detection\anomalib\data\multi_category_mvtec.py:227: UserWarning: seed is None. When seed is not set, images from the normal directory are split between training and test dir. This will lead to inconsistency between runs.
  warnings.warn(


In [8]:
for epoch in range(10):
    model.train()
    train_loss_sum = 0.0
    train_acc_sum = 0.0
    train_n = 0
    test_acc_sum = 0.0
    test_n = 0
    for data in datamodule.train_dataloader():
        X = data['image'].cuda()
        y = data['category_label'].cuda()
        y_hat = model(X)
        l = loss(y_hat, y).sum()
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        train_loss_sum += l.item()
        train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
        train_n += y.shape[0]
    lr_scheduler.step()
    model.eval()

    for data in datamodule.test_dataloader():
        X = data['image'].cuda()
        y = data['category_label'].cuda()
        test_acc_sum += (model(X).argmax(dim=1) == y).float().sum().item()
        test_n += y.shape[0]

    print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f' %
          (epoch + 1, train_loss_sum / train_n, train_acc_sum / train_n, test_acc_sum / test_n))
    torch.save(model,f'../models/mobilenet_v2_epoch_{epoch + 1}.pth')

epoch 1, loss 0.1113, train acc 0.341, test acc 0.237
epoch 2, loss 0.0289, train acc 0.732, test acc 0.586
epoch 3, loss 0.0161, train acc 0.856, test acc 0.724
epoch 4, loss 0.0109, train acc 0.900, test acc 0.795
epoch 5, loss 0.0062, train acc 0.937, test acc 0.823
epoch 6, loss 0.0072, train acc 0.936, test acc 0.535
epoch 7, loss 0.0039, train acc 0.968, test acc 0.977
epoch 8, loss 0.0032, train acc 0.967, test acc 0.956
epoch 9, loss 0.0023, train acc 0.980, test acc 0.987
epoch 10, loss 0.0013, train acc 0.987, test acc 0.962


In [9]:
model = torch.load("../models/mobilenet_v2_epoch_9.pth")
model

MobileNetV2(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05,